In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset_train = pd.read_csv('carga_gw_2020.csv', sep = ';', engine = "python", header = None)


In [3]:
def to_numeric(x): 
    
    x = x.replace(',','.')
    sep = x.split('.')
    
    if len(sep) == 2:
        x = sep[0]+'.'+sep[1]
        
        
    elif len(sep) ==3 :
        x = sep[0]+sep[1]+'.'+sep[2]
    
    else:
        x = 0
    
    x_num = pd.to_numeric(x)

    return  x_num


In [4]:
def preprocess_df(df):
        
    # Determina novos nome de colunas
    df.columns = ["Date", "Carga"]
    
    df.Date = pd.to_datetime(df.Date, dayfirst= True)
    df.dtypes
    
    # Transforma colunas em numericos 
    df['Carga'] = pd.to_numeric(df['Carga'].apply(to_numeric))
    
    
    
    # Associa coluna Date ao index
    #df.index = pd.to_datetime(df.Date, format="%m-%d-%Y")
    # dropa coluna Date extra 
    #df.drop("Date", axis=1, inplace=True)
    
    return df

In [5]:
def fill_na_with_mean(df):
    
    num_cols = len(list(df.columns.values))
    
    for col in range(num_cols):
        
        df.iloc[:,col] = df.iloc[:,col].fillna(df.iloc[:,col].mean())
        
    return df

In [6]:
dataset_train = preprocess_df(dataset_train)
dataset_train = fill_na_with_mean(dataset_train)
    
dataset_train.head()

,Date,Carga
0,2018-01-01,1264.5
1,2018-01-02,1519.4
2,2018-01-03,1591.3
3,2018-01-04,1588.7
4,2018-01-05,1594.7


In [7]:
training = dataset_train.iloc[: , 1:2]

In [8]:
train = training[0:729]
train.shape

(729, 1)

##Feature scaling

In [9]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
train_scaled = sc.fit_transform(train)

##Criando uma estrututra de dados que receba 60 passos no tempo e retorne uma saída

In [10]:
x_train = []
y_train = []
for i in range(60,729): 
    x_train.append(train_scaled[i-60:i, 0])
    y_train.append(train_scaled[i, 0])
x_train, y_train = np.array(x_train), np.array(y_train) 

#Reshaping 

In [11]:
x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1], 1))

In [14]:
from keras.models import Sequential 
#from keras.layers import Dense
#from keras.layers import LSTM
#from keras.layers import Dropout

ImportError: Keras requires TensorFlow 2.2 or higher. Install TensorFlow via `pip install tensorflow`

In [46]:
# Inicializando a RNN
regressor = Sequential()

NameError: name 'Sequential' is not defined

##Adicionando a primeira layer de LSTM e uma camada de dropout

In [ ]:
regressor.add(LSTM(units=50, return_sequences = True, input_shape = (x_train.shape[1],1)))
regressor.add(Dropout(0.2))

## Adicionando a segunda camada 

In [ ]:
regressor.add(LSTM(units=50, return_sequences = True,))
regressor.add(Dropout(0.2))

## Adicionado a terceira camada 

In [ ]:
regressor.add(LSTM(units=50, return_sequences = True))
regressor.add(Dropout(0.2))

## Adicionando a quarta camada 

In [ ]:
regressor.add(LSTM(units=50))
regressor.add(Dropout(0.2))

## Adicionando a camada de saída 

In [ ]:
regressor.add(Dense(units=1))

In [ ]:
regressor.compile(optimizer = 'adam', Loss = 'mean_squared_error')

In [ ]:
regressor.fit(x_train, y_train, epochs = 100, batch_size = 32)

In [ ]:
test = training[729:912]

In [ ]:
total = np.concatenate((train, test))